# Ejercicio: Análisis de grupos de Whatsapp
**Autor**: Fermín Cruz.   **Revisor**: José A. Troyano, J. Mariano González, Carlos G. Vallejo, Beatriz Pontes.  **Última modificación**: 04/12/2018


En este ejercicio vamos a analizar los mensajes de un grupo de Whatsapp. En concreto, podremos obtener toda esta información:
* Número de mensajes a lo largo del tiempo
* Número de mensajes según el día de la semana, o la hora del día
* Número de mensajes escritos por cada participante en el grupo
* Palabras más utilizadas en el grupo
* Palabras más características de cada participante en el grupo

Para llevar a cabo el ejercicio, necesitamos un archivo de log de un grupo de Whatsapp. Junto a este notebook encontrarás un fichero de log ficticio que puedes utilizar para desarrollar los ejercicios (generado a partir de los diálogos de la primera temporada de The Big Bang Theory); sin embargo, es mucho más divertido si utilizas un log de un grupo propio, si es que eres usuario de Whatsapp. Para ello, sigue los siguientes pasos, según estés en **Android** o en **iOS** (*estos pasos han sido probado con las últimas versiones de Whatsapp a día 11 de diciembre de 2017, puede que con otras versiones los pasos cambien ligeramente*):

* Android:
 * Entra en Whatsapp en tu móvil, y accede al grupo que desees. Cuantos más mensajes haya en el grupo, tanto más interesante será el análisis que llevaremos a cabo.
 * Pulsa en el menú contextual, y busca la opción "Enviar chat por correo". Selecciona la opción "Sin archivo(s)".
 * Escribe como destinatario tu dirección de email.
 * Accede al email desde este ordenador, y descarga el fichero cuyo nombre acaba en txt.
 * Guarda este archivo dentro de la carpeta datos de este notebook. Quizás prefieras cambiarle el nombre al fichero por uno más corto.
 
 
* iOS:
 * Entra en Whatsapp en tu móvil, y accede al grupo que desees.
 * Pulsa sobre el nombre del grupo, en la parte superior. Busca la opción "Exportar chat".
 * Selecciona "Sin archivos", y luego "Compartir mediante correo electrónico". Escribe tu dirección de correo.
 * Accede al email desde este ordenador, y descarga el fichero cuyo nombre acaba en txt.
 * Guarda este archivo dentro de la carpeta datos de este notebook. Quizás prefieras cambiarle el nombre al fichero por uno más corto.

El el formato del fichero que genera Whatsapp varía según se trate de la versión Andriod o iOS. Veamos un ejemplo de cada caso:
* Android:
<pre>
26/02/16, 09:16 - Leonard: De acuerdo, ¿cuál es tu punto?
26/02/16, 16:16 - Sheldon: No tiene sentido, solo creo que es una buena idea para una camiseta.
</pre>
* iOS:
<pre>
[26/2/16 09:16:25] Leonard: De acuerdo, ¿cuál es tu punto?
[26/2/16 16:16:54] Sheldon: No tiene sentido, solo creo que es una buena idea para una camiseta.
</pre>

Como puedes observar, cada mensaje del chat viene precedido por la fecha, la hora, y el nombre del usuario. El formato no es CSV, como en otros ejercicios, sino que es, digamos, más libre. Así, la fecha ocupa el principio de una línea, que puede venir seguida de una coma. Tras un espacio en blanco, viene la hora (que puede contener o no los segundos), seguida de un espacio, un guión y otro espacio, o bien de dos puntos y un espacio, según la versión de Whatsapp. Por último, tenemos el nombre del usuario, acabando en dos puntos, y tras otro espacio, aparece el texto del mensaje.

---
Como siempre, primero importemos los módulos que necesitamos. 

In [ ]:
import datetime
from datetime import timedelta
import re
from collections import namedtuple, Counter
import matplotlib.pyplot as plt

Si vas a usar tu propio log, cambia el nombre del fichero en la variable FICHERO. Por cierto, ¿has observado que el nombre de esta variable está entero en mayúsculas? Es la manera habitualmente utilizada en Python para indicar que el valor de dicha variable no va a cambiar a lo largo de la ejecución, es decir, que va a permanecer **constante**.

In [ ]:
FICHERO = 'data/bigbangtheory_es.txt'

# Puedes cambiar estos valores para variar el tamaño de las figuras generadas (ancho, alto)
plt.rcParams["figure.figsize"] = [10, 8]

# Cambia este valor para variar el número de palabras mque se mostrarán en las nubes de palabras
NUM_PALABRAS_NUBE = 100

# 1. Lectura del fichero de log

En la siguiente función hay que implementar la lectura del fichero de log. Como el formato del fichero es menos regular que en ejercicios anteriores, habrá que leerlo línea a línea y analizar estas líneas para extraer los distintos campos de información (fecha, hora, usuario y texto de cada mensaje). La mejor manera de extraer esta información es mediante el uso de **expresiones regulares**. Estas expresiones nos permiten definir patrones en los que pueden encajar distintos trozos de texto, y extraer información a partir de distintos trozos de la cadena que ha sido reconocida.

Vamos a usar una expresión regular para la versión Android de los logs y otra para la versión iOS:

In [ ]:
ANDROID_RE = r'(\d\d?/\d\d?/\d\d?) (\d\d?:\d\d) - ([^:]+): (.+)'
IOS_RE = r'\[(\d\d?/\d\d?/\d\d?) (\d\d?:\d\d):\d\d\] ([^:]+): (.+)'

Fijémonos en la versión Android para ver qué significa:

* La ```r``` que precede a cadena indica a Python que se trata de una expresión regular.
* Cada uno de los tramos encerrados entre paréntesis se corresponde con un dato que vamos a extraer. Verás que en total hay cuatro tramos:
 * El primer tramo de la cadena, ```(\d\d?/\d\d?/\d\d?)```, es un patrón que encaja con las fechas que aparecen en los mensajes. Los \d indican dígitos, y las interrogaciones indican partes que pueden aparecer o no.
 * El segundo tramo, ```(\d\d?:\d\d)```, encaja con las horas.
 * El tercer tramo, ```([^:]+)```, reconoce los nombres de usuario. La expresión ```[^:]+``` significa *"cualquier combinación de caracteres salvo los dos puntos"* (es decir, que reconoce todo el trozo de texto que viene antes de los dos puntos).
 * Y el último tramo, ```(.+)```, captura el texto de los mensajes. 

Como es la primera vez que aparecen expresiones regulares, esta función se da parcialmente resuelta. 

In [ ]:
Mensaje = namedtuple('Mensaje', 'fecha hora usuario texto')

def carga_log(fichero, os='android', debug=False):
    ''' Carga un log de Whatsapp, devolviéndolo como lista de tuplas.
    
    ENTRADA: 
       - fichero: nombre del fichero del que se quieren leer los datos -> str
       - debug: indica si se desea obtener información sobre la carga -> bool
    SALIDA: 
       - lista de mensajes -> [Mensaje(datetime.date, datetime.time, str, str)] 
    
    Si el parámetro debug es True se mostrarán los usuarios y el intervalo de 
    fechas procesado. Por ejemplo: 
        3779 mensajes leídos.
        Usuarios: {'Penny', 'Sheldon', 'Howard', 'Raj', 'Lesley', 'Leonard'}
        Intervalo de fechas: 2016-02-25 -> 2017-03-04
    
    La función devuelve una lista de tuplas, cada una de ellas conteniendo la fecha,
    la hora, el usuario y el texto de un mensaje. El orden de las tuplas en la lista
    es el mismo que el que aparece en el fichero, es decir, cronológico.
    '''
    if os=='android':
        regex = ANDROID_RE
    elif os=='ios':
        regex = IOS_RE
    else:
        raise Exception('OS no permitido')
        
    log = []
    with open(fichero, encoding='utf8') as f:        
        for linea in f:
            # Aplicamos la expresión regular sobre cada línea
            matches = re.findall(regex, linea)
            if matches:  # Si se encuentran coincidencias para los patrones
                fecha_str, hora_str, usuario, texto = matches[0]
                fecha = datetime.datetime.strptime(fecha_str, '%d/%m/%y').date()
                hora = datetime.datetime.strptime(hora_str, '%H:%M').time()
                log.append(Mensaje(fecha,hora,usuario, texto))
    if debug:
        pass
        
    return log

Probemos la función anterior. La salida esperada es:
<pre>
3726 mensajes leídos.
Usuarios: ['Howard', 'Leonard', 'Penny', 'Raj', 'Sheldon']
Intervalo de fechas: 2016-02-25 -> 2017-03-04
</pre>

In [ ]:
# Test de la función carga_log
log = carga_log(FICHERO, debug=True)

# 2. Análisis basado en la distribución del número de mensajes

Los primeros análisis que vamos a realizar están relacionados con el número de mensajes y su distribución en relación a las fechas, las horas y los usuarios. 

## 2.1. Construcción de índices

Para llevar a cabo esta tarea, nos será muy útil contar con diccionarios que nos agrupen los mensajes ocurridos en una misma fecha, una misma hora o escritos por un mismo usuario. Implemente la siguiente función, que indexa los mensajes por el usuario.

In [ ]:
def indexa_por_usuario(log):
    ''' Calcula un diccionario indexado por nombre de usuario
    
    ENTRADA: 
       - log: lista de mensajes -> [Mensaje(datetime.date, datetime.time, str, str)]
    SALIDA: 
       - diccionario indexado por usuarios -> {str: [Mensaje(datetime.date, datetime.time, str, str)]} 
    
    Para cada usuario, el diccionario contendrá la lista de mensajes escritos por él. 
    '''
    pass

Probemos la función, cuya salida esperada es:
<pre>
Mensajes para  Howard : 428
Primer mensaje de Howard : Mensaje(fecha=datetime.date(2016, 3, 19), hora=datetime.time(9, 4), usuario='Howard', texto='Espera a ver esto.')
Mensajes para  Leonard : 1210
Primer mensaje de Leonard : Mensaje(fecha=datetime.date(2016, 2, 26), hora=datetime.time(9, 16), usuario='Leonard', texto='De acuerdo, ¿cuál es tu punto?')
Mensajes para  Penny : 712
Primer mensaje de Penny : Mensaje(fecha=datetime.date(2016, 3, 1), hora=datetime.time(12, 30), usuario='Penny', texto='¡Oh hola!')
Mensajes para  Raj : 251
Primer mensaje de Raj : Mensaje(fecha=datetime.date(2016, 3, 19), hora=datetime.time(9, 11), usuario='Raj', texto='Es fantástico. Increíble.')
Mensajes para  Sheldon : 1125
Primer mensaje de Sheldon : Mensaje(fecha=datetime.date(2016, 2, 25), hora=datetime.time(22, 33), usuario='Sheldon', texto='Entonces, si un fotón se dirige a través de un plano con dos rendijas y se observa que no se pasará por ambas rendijas. Si no se observa, lo hará, sin embargo, si se observa después de que ha salido del avión, pero antes de que llegue a su objetivo, no habrá pasado por las dos rendijas.')
</pre>

In [ ]:
indice_por_usuario = indexa_por_usuario(log)
for usuario in sorted(indice_por_usuario):
    print("Mensajes para ", usuario,":", len(indice_por_usuario[usuario]))
    print("Primer mensaje de", usuario,":", indice_por_usuario[usuario][0])

Si ahora queremos obtener índices por fecha y por hora, podríamos copiar el código de la función indexa_por_usuario, cambiando en el cuerpo de la función únicamente el acceso al campo usuario por un acceso al campo fecha u hora. Siendo conscientes de esto, parece más adecuado implementar una única función, indexa_por_campo, que parametrice el nombre del campo de las tuplas que se desea usar como clave en el diccionario devuelto. Así es como lo vamos a hacer:

In [ ]:
def indexa_por_campo(log, campo):
    ''' Calcula un diccionario indexado por un campo
    
    ENTRADA: 
       - log: lista de mensajes -> [Mensaje(datetime.date, datetime.time, str, str)]
       - campo: nombre del campo por el que se quiere indexar -> str
    SALIDA: 
       - diccionario indexado por el campo -> {<tipo del campo>: [Mensaje(datetime.date, datetime.time, str, str)]}
       
    PISTA: ¿Cómo se accede al campo de una namedtupla cuando tenemos el nombre del campo en una variable?
    '''
    pass

Podemos probar la función para obtener el mismo índice por usuario anterior (también puedes hacer tus propias pruebas para ver qué se obtiene al indexar por otros campos):

In [ ]:
indice_por_usuario = indexa_por_campo(log, 'usuario')
for usuario in sorted(indice_por_usuario):
    print("Mensajes para ", usuario,":", len(indice_por_usuario[usuario]))
    print("Primer mensaje de", usuario,":", indice_por_usuario[usuario][0])

## 2.2. Construcción de las gráficas

Vamos ahora a dibujar gráficas que nos permitan visualizar cómo se distribuyen los mensajes con respecto a las fechas, las horas o los usuarios. Empecemos dibujando una gráfica de barras que muestre **el número de mensajes que ha escrito cada uno de los usuarios del chat**. 

PISTA: Es posible que necesites usar la función predefinida **zip**, junto con el operador \*, que sirve para invertir el funcionamiento habitual de *zip*. Puedes ver un ejemplo aquí: https://stackoverflow.com/a/12974504

In [ ]:
def dibuja_mensajes_por_usuario(log):
    ''' Muestra una gráfica de barras indicando el número de mensajes por cada usuario 
    
    ENTRADA: 
       - log: lista de mensajes -> [Mensaje(datetime.date, datetime.time, str, str)]
    SALIDA EN PANTALLA: 
       - diagrama de barras con el número de mensajes por cada usuario
       
    Para dibujar la gráfica usaremos las siguientes instrucciones matplotlilb:
        plt.barh(range(len(mensajes)), mensajes, tick_label=usuarios)
        plt.show()
    
    Y necesitaremos calcular las siguientes variables:
       - usuarios = lista de usuarios que han escrito mensajes, ordenados alfabéticamente
       - mensajes = lista (alineada con la anterior) con el número de mensajes escritos por cada usuario
    '''
    pass

Probemos la función anterior. El resultado debe ser parecido a éste:
![title](img/barras_usuarios.png)

In [ ]:
# Test de dibuja_mensajes_por_usuario
dibuja_mensajes_por_usuario(log)

Ahora queremos dibujar una gráfica de barras que muestre el **número de mensajes a lo largo del tiempo**. Para que la información sea más fácil de interpretar, agregaremos los mensajes por meses, mostrando el número total de mensajes en cada uno de los meses entre las fechas inicial y final del log. 



In [ ]:
def dibuja_mensajes_por_meses(log):
    ''' Muestra una gráfica de barras indicando el número de mensajes mensualmente
    
    ENTRADA: 
       - log: lista de mensajes -> [Mensaje(datetime.date, datetime.time, str, str)]
    SALIDA EN PANTALLA: 
       - diagrama de barras vertical con el número de mensajes por cada mes
       
    Para dibujar la gráfica usaremos las siguientes instrucciones matplotlilb:
        plt.bar(range(len(mensajes)), mensajes)
        plt.xticks(range(len(meses)), meses, rotation=80, fontsize=10)
        plt.show()
    
    Y necesitaremos calcular las siguientes variables:
       - meses = lista de meses en los que se han escrito mensajes
       - mensajes = lista (alineada con la anterior) con el número de mensajes escritos cada mes
    
    PISTA: Lo más sencillo es generar una lista con cadenas "mes/año" de cada mensaje del log, y luego
    contar las apariciones de cada cadena de dicha lista usando Counter.
    '''    
    pass

Probemos la función anterior. El resultado debe ser parecido a éste:
![title](img/barras_meses.png)

In [ ]:
# Test de la función muestra_info_por_fechas
dibuja_mensajes_por_meses(log)

Por último, dibujemos dos gráficas más:
* Una gráfica que muestre el **volumen de mensajes según el día de la semana** (de lunes a domingo).
* Una gráfica que muestre el **volumen de mensajes según la hora del día** (de 0 a 23).

In [ ]:
def dibuja_mensajes_agregados_por_dias_semana(log):
    ''' Muestra una gráfica de barras indicando el número de mensajes agregados por día de la semana
    
    ENTRADA: 
       - log: lista de mensajes -> [Mensaje(datetime.date, datetime.time, str, str)]
    SALIDA EN PANTALLA: 
       - diagrama de barras vertical con el número de mensajes por cada día de la semana
       
    Para dibujar la gráfica usaremos las siguientes instrucciones matplotlilb:
        plt.bar(range(len(dias)), mensajes, tick_label=dias)
        plt.show()
    
    Y necesitaremos calcular las siguientes variables:
       - dias = lista de dias de la semana ['L','M','X','J','V','S','D']
       - mensajes = lista (alineada con la anterior) con el número de mensajes escritos cada dia
    '''
    pass

Probemos la función anterior. El resultado debe ser parecido a éste:
![title](img/barras_dias.png)

In [ ]:
# Test de la función dibuja_mensajes_agregados_por_dias_semana
dibuja_mensajes_agregados_por_dias_semana(log)

In [ ]:
def dibuja_mensajes_agregados_por_horas(log):
    ''' Muestra una gráfica de barras indicando el número de mensajes agregados por horas
    
    ENTRADA: 
       - log: lista de mensajes -> [Mensaje(datetime.date, datetime.time, str, str)]
    SALIDA EN PANTALLA: 
       - diagrama de barras vertical con el número de mensajes por cada hora del día
       
    Para dibujar la gráfica usaremos las siguientes instrucciones matplotlilb:
        plt.bar(range(len(horas)), mensajes, tick_label=horas)
        plt.show()
    
    Y necesitaremos calcular las siguientes variables:
       - horas = lista de horas del día (de 0 a 23)
       - mensajes = lista (alineada con la anterior) con el número de mensajes escritos cada hora
    '''    
    pass

Probemos la función anterior. El resultado debe ser parecido a éste:
![title](img/barras_horas.png)

In [ ]:
# Test dibuja_mensajes_agregados_por_horas
dibuja_mensajes_agregados_por_horas(log)

Ahora puedes jugar a sacar gráficas combinando las funciones anteriores y el uso de la función indice_por_campo. Por ejemplo, podemos obtener un gráfico que muestre la actividad del usuario 'Sheldon' a lo largo de la semana, de esta forma:

In [ ]:
# Cambia el usuario por uno que aparezca en el log que estés utilizando
dibuja_mensajes_agregados_por_dias_semana(indice_por_usuario['Sheldon'])

Otros gráficos que puedes intentar generar:
* Mensajes por hora de un usuario determinado
* Mensajes a lo largo del tiempo de un usuario determinado
* Mensajes por usuario en una fecha concreta
* Mensajes por horas en una fecha concreta

# 3. Análisis basado en el contenido textual de los mensajes

En estos ejercicios nos vamos a centrar en los textos de los mensajes. Haremos un análisis muy superficial de estos textos, basado principalmente en la frecuencia de aparición de las palabras. Este tipo de procedimientos se emplean en la disciplina conocida como **Minería de Textos**; otro tipo de análisis más profundos se llevan a acabo en la disciplina conocida como **Procesamiento del Lenguaje Natural**, aunque no entraremos en ellos.

Para realizar estos ejercicios utilizaremos un módulo llamado **WordCloud**, que es un desarrollo propio del profesor **Andreas Mueller** (*Columbia University*). Para instalarlo en tu sistema y poder utilizarlo:

* En primer lugar, cierra la ventana en la que estás ejecutando *jupyter notebook*. Puedes hacer *Ctrl+C* repetidas veces para abortar la ejecución.

* Si estás en Windows, abre Anaconda Prompt, navega hasta la carpeta donde se encuentra este notebook, y ejecuta este comando:
<pre>
pip install fichero_whl
</pre>
, donde *fichero_whl* puede ser *wordcloud-1.3.2-cp36-cp36m-win_amd64.whl* o *wordcloud-1.3.2-cp36-cp36m-win32.whl*, según tengas instalada la versión de 64 bits o la de 32 bits de Anaconda. Si no estás seguro, prueba con la de 64 bits.

* Si estás en Windows, pero lo anterior te da un error, es posible que necesites otra versión del fichero de extensión whl. Puedes encontrar otras versiones aquí: https://www.lfd.uci.edu/~gohlke/pythonlibs/#wordcloud. Si no tienes claro qué versión necesitas, habla con tu profesor.

* Si estás en otro sistema operativo (Linux, MacOs), sigue las instrucciones que aparecen aquí: https://github.com/amueller/word_cloud#installation.

* Una vez hayas acabado la instalación, vuelve a ejecutar *jupyter notebook*.

## 3.1. Nubes de palabras frecuentes con WordCloud

En primer lugar, implemente la función *muestra_word_cloud*, que muestra una *nube de palabras* con los términos más frecuentemente utilizados en nuestro log. 

In [ ]:
from wordcloud import WordCloud

def muestra_word_cloud(log):
    ''' Muestra una nube de palabras a partir de todo el texto del log  
    
    ENTRADA: 
       - log: lista de mensajes -> [Mensaje(datetime.date, datetime.time, str, str)]
    SALIDA EN PANTALLA: 
       - nube de palabras
       
    Para obtener el gráfico en cuestión, utilizar el siguiente código:
        wordcloud = WordCloud(
                 font_path='datos/CabinSketch-Bold.ttf',
                 background_color='white',
                 width=1800,
                 height=1400,
                 normalize_plurals=False,
                 max_words=200
                 ).generate(texto)
        plt.imshow(wordcloud)
        plt.axis('off')
        plt.show()

    La variable que hay que calcular es:
       - texto: una cadena que contiene todo el texto de todos los mensajes del log
    '''
    pass

Si probamos la función anterior, deberíamos obtener un gráfico parecido a éste:
![title](img/word_cloud_1.png)

In [ ]:
muestra_word_cloud(log)

Las nubes de palabras pretenden representar en una sola imagen la información más importante contenida en un texto. Sin embargo, en el gráfico que acabamos de obtener, la mayoría de las palabras que aparecen no nos dan demasiada información sobre el contenido del log. Palabras como "en", "que", "un", "es", ..., son muy frecuentes, pero poco informativas. 

A este tipo de palabras se las conoce como *palabras huecas* (en inglés, *stop words*), y lo ideal sería que las filtrásemos a la hora de construir el grafo. Complete la implementación de la siguiente función, que devuelve un conjunto con las palabras leídas de un fichero de entrada. Tenga en cuenta que el fichero que va a leer usa una codificación de caracteres **utf-8**.

In [ ]:
def carga_palabras_huecas(fichero='data/palabras_huecas.txt'):
    ''' Devuelve un conjunto con las palabras leídas del fichero 
    
    ENTRADA: 
       - fichero: nombre del fichero en el que se encuentra la lista de palabras huecas -> str
    SALIDA: 
       - conjunto de palabras huecas -> {str}

    El fichero tiene una palabra por línea.
    '''
    pass

Si probamos la función, debemos obtener un conjunto con todas las palabras leídas del fichero.

In [ ]:
# Test de carga_palabras_huecas
palabras_huecas = carga_palabras_huecas()
print(len(palabras_huecas), "palabras huecas cargadas.")
print(sorted(palabras_huecas))

Ahora podemos modificar la función *muestra_word_cloud*, para que reciba un parámetro con las palabras que se desea ignorar. Añada el conjunto de palabras en cuestión al parámetro de nombres *stopwords* cuando invoque a la función *WordCloud*.

In [ ]:
def muestra_word_cloud(log, ignora_palabras=set()):
    texto = ' '.join((mensaje.texto for mensaje in log))    
    wordcloud = WordCloud(
                      font_path='data/CabinSketch-Bold.ttf',
                      stopwords=ignora_palabras,
                      background_color='white',
                      width=1800,
                      height=1400,
                      normalize_plurals=False,
                      max_words=NUM_PALABRAS_NUBE
                     ).generate(texto)
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

Si probamos la función anterior, deberíamos obtener un gráfico con palabras más informativas que antes, parecido a éste:
![title](img/word_cloud_2.png)

Si sigues viendo palabras que crees que no deberían aparecer porque no aportan información, puedes añadirlas al fichero de palabras huecas y volver a generar el gráfico.

In [ ]:
muestra_word_cloud(log, palabras_huecas)

El mismo procedimiento podemos combinarlo con el uso del índice por usuarios para obtener nubes de palabras para uno de los usuarios de nuestro log:

In [ ]:
usuario = 'Sheldon'  # Si utilizas tu propio log, cambia esto por algún usuario de tu grupo
print("Nube de palabras de",usuario)
muestra_word_cloud(indice_por_usuario[usuario], palabras_huecas)

## 3.2. Nubes de palabras características de usuarios

Si observas las nubes generadas, verás que hay algunas palabras que son características de cada usuario, mientras que muchas otras aparecen en todas las nubes o en la mayoría de ellas. ¿Podríamos generar un gráfico que evite las palabras comunes, y que contenga únicamente palabras características de cada usuario? Esto nos puede proporcionar más información sobre las particularidades de los mensajes de cada usuario.

Para poder hacer esto, tenemos que generar la lista de palabras a representar junto con un número por cada palabra que indique el tamaño que queremos que tenga la palabra en el gráfico. Vamos a comenzar generando **diccionarios en los que aparezcan todas las palabras de un usuario junto al número de apariciones de dichas palabras en los textos de sus mensajes**.

In [ ]:
def genera_conteos_palabras_por_usuario(log):
    ''' Genera un diccionario con el conteo de las palabras usadas por cada usuario.
    
    ENTRADA: 
       - log: lista de mensajes -> [Mensaje(datetime.date, datetime.time, str, str)]
    SALIDA: 
       - diccionario de conteos de palabras por cada usuario -> {str: {str: int}}
    
    Por ejemplo, el diccionario devuelto podría ser así:
    {'usuario1':{'palabra1':5, 'palabra2':14, 'palabra3':7, ...},
     'usuario2':{'palabra1':6, 'palabra2':1, 'palabra3':12, ...},
     'usuario3':{'palabra1':3, 'palabra2':7, 'palabra3':18, ...},
      ...
    }
    
    Para evitar que aparezcan signos de puntuación, aplicar a cada palabra
    la siguiente instrucción:
        palabra = palabra.strip(".,:();¿?¡!")
    '''
    pass

Vamos a probar la función. Deberías obtener una salida parecida a ésta:
<pre>
Howard
    Espera -> 9
    a -> 103
    ver -> 4
    esto -> 11
    Es -> 10
   ...
Leonard
    De -> 46
    acuerdo -> 31
    cuál -> 2
    es -> 187
    tu -> 50
   ...
Penny
    Oh -> 79
    hola -> 5
    Hola -> 29
    eso -> 38
    es -> 122
   ...
Raj
    Es -> 4
    fantástico -> 1
    Increíble -> 3
    Por -> 17
    qué -> 23
   ...
Sheldon
    Entonces -> 28
    si -> 48
    un -> 254
    fotón -> 1
    se -> 120
   ...
</pre>

In [ ]:
# Test de la función 
test_conteos = genera_conteos_palabras_por_usuario(log)
for usuario, conteos in sorted(test_conteos.items()):
    print(usuario)
    for palabra, conteo in list(conteos.items())[:5]:
        print('   ',palabra,'->',conteo)
    print('   ...')

Lo que acabamos de hacer es básicamente lo que hace automáticamente WordCloud cuando le pasamos un texto: contar las apariciones de las palabras, y dibujar la nube a partir de dicha información. Para quedarnos con las palabras características de cada usuario, vamos a utilizar una fórmula que disminuya el valor asociado a aquellas palabras que son usadas por el resto de usuarios. Antes de plantear dicha fórmula y realizar el cálculo, necesitamos realizar la siguiente función, que para un usuario determinado nos devuelve dos diccionarios:
* Uno con los conteos de las palabras usadas por el usuario.
* Otro con los conteos totales de las palabras usadas por el resto de usuarios.

Para entenderlo mejor, fijémonos en una única palabra, llamémosla *p*. Si en el primer diccionario aparece la palabra *p* asociada al número 23, significa que el usuario en cuestión la ha usado 23 veces en sus mensajes. Si la misma palabra *p* aparece asociada al número 104 en el segundo diccionario, significa que dicha palabra ha sido usada un total de 104 veces por el resto de usuarios. 

Ambos diccionarios se pueden construir partiendo de la información que nos genera la funcion *genera_conteos_palabras_por_usuario*.

In [ ]:
def genera_conteos_palabra_usuario_y_resto(log, usuario):
    ''' Calcula conteo de palabras globales y de un usuario
    
    ENTRADA: 
       - log: lista de mensajes -> [Mensaje(datetime.date, datetime.time, str, str)]
       - usuario: del que se calculará el conteo específico
    SALIDA: 
       - diccionario de conteos de palabras del usuario -> {str: int}
       - diccionario con la suma de conteos de palabras del resto de usuarios -> {str:int}
    '''
    pass

Probemos la función. Debes obtener una salida parecida a ésta:
<pre>
Usuario: Sheldon
Entonces -> 28
	conteo para el resto de usuarios: 77
si -> 48
	conteo para el resto de usuarios: 97
un -> 254
	conteo para el resto de usuarios: 383
fotón -> 1
	palabra no usada por el resto de usuarios.
se -> 120
	conteo para el resto de usuarios: 163
</pre>

In [ ]:
# Test de la función genera_conteos_palabra_usuario_y_resto
usuario = 'Sheldon' # Cámbialo por algún usuario que aparezca en tu log, si es preciso
frecuencias_usuario, frecuencias_resto = genera_conteos_palabra_usuario_y_resto(log, usuario)
print('Usuario:', usuario)
for palabra, conteo in list(frecuencias_usuario.items())[:5]:
    print(palabra,'->',conteo)
    if palabra in frecuencias_resto:
        print('\tconteo para el resto de usuarios:',frecuencias_resto[palabra])
    else:
        print('\tpalabra no usada por el resto de usuarios.')

Ahora estamos en disposición de generar el diccionario de palabras características de un usuario. Recordemos que, para cada palabra, queremos obtener un número que sea proporcional al número de veces que el usuario ha usado esa palabra, y al mismo tiempo inversamente proporcional al número de veces que esa palabra ha sido usada por el resto de usuarios. 

Si llamamos *importancia* a este valor que asociamos a las palabras, la fórmula que usaremos será la siguiente:

$$
importancia_{usuario}(palabra) = \frac{conteo_{usuario}(palabra)}{total\_palabras_{usuario}}*\frac{total\_palabras_{resto}}{conteo_{resto}(palabra)}$$

Donde:
* $conteo_{usuario}(palabra)$ es el número de veces que $usuario$ ha usado la palabra $palabra$ en sus mensajes.
* $conteo_{resto}(palabra)$ es el número de veces que el resto de usuarios han usado la palabra $palabra$ en sus mensajes.
* $total\_palabras_{usuario}$ es el total de palabras de todos los mensajes escritos por el usuario.
* $total\_palabras_{usuario}$ es el total de palabras de todos los mensajes escritos por el resto de usuarios.

Si te fijas, la primera parte de la fórmula indica el ratio (de 0 a 1) del número de apariciones de una palabra en los mensajes del usuario con respecto al total de palabras de sus mensajes. La segunda parte de la fórmula es la inversa del ratio del número de apariciones de la misma palabra en los mensajes del resto de usuarios con respecto al total de palabras de sus mensajes. Así, las palabras con un alto ratio de uso por parte del usuario y un bajo ratio de uso por parte del resto serán las que obtengan una mayor importancia (y serán representadas con mayor tamaño en la nube de palabras). Esta forma de ponderar la importancia de las palabras se parece mucho a la medida [tf-idf](https://es.wikipedia.org/wiki/Tf-idf), usada en distintas técnicas de [recuperación de información](https://es.wikipedia.org/wiki/B%C3%BAsqueda_y_recuperaci%C3%B3n_de_informaci%C3%B3n) y [minería de textos](https://es.wikipedia.org/wiki/Miner%C3%ADa_de_textos).

Para que el denominador nunca sea 0, si $conteo_{resto}(palabra)$ es igual a 0 para una palabra determinada (es decir, la palabra no ha sido usada por ningún otro usuario), entonces usaremos $conteo_{resto}(palabra) = 0.00001$.

In [ ]:
def genera_palabras_caracteristicas_usuario(log, usuario, umbral=2):
    ''' Genera un diccionario con la importancia de las palabras usadas por un usuario
    
    ENTRADA: 
       - log: lista de mensajes -> [Mensaje(datetime.date, datetime.time, str, str)]
       - usuario: del que se calculará la importancia de las palabras
       - umbral: frecuencia mínima para tener en cuenta una palabra -> int
    SALIDA: 
       - diccionario de importancia de las palabras del usuario -> {str: float}
       - diccionario con la suma de conteos de palabras del resto de usuarios -> {str:int}
    
    La importancia de las palabras se define como un valor proporcional al
    número de veces que el usuario la ha usado, e inversamente proporcional
    al número de veces que ha sido usada por el resto de usuarios.
    
    El parámetro umbral permite filtrar palabras que hayan sido usadas pocas 
    veces, de manera que la importancia asociada a dichas palabras en el 
    diccionario devuelto será 0.
    '''
    pass

Probemos la función anterior. El resultado debe ser parecido a éste:
<pre>
Usuario: Sheldon
    Entonces -> 0.620867905841999
    si -> 0.8448924079499369
    un -> 1.1323139222732022
    fotón -> 0
    se -> 1.2569718339132496
</pre>

In [ ]:
# Test de la función genera_palabras_caracteristicas_usuario
usuario = 'Sheldon' # Cámbialo por algún usuario que aparezca en tu log, si es preciso
importancia_usuario = genera_palabras_caracteristicas_usuario(log, usuario)
print('Usuario:', usuario)
for palabra, importancia in list(importancia_usuario.items())[:5]:
        print('   ',palabra,'->',importancia)

Sólo nos queda mostrar la información anterior mediante una nube de palabras. Esto se hace mediante el método *generate_from_frequencies*. La función consiste simplemente en construir la imagen mediante dicho método, así que se proporciona solucionada:

In [ ]:
def muestra_word_cloud_por_frecuencias(frecuencias, ignora_palabras=set()):
    wordcloud = WordCloud(
                      font_path='data/CabinSketch-Bold.ttf',
                      stopwords=ignora_palabras,
                      background_color='white',
                      width=1800,
                      height=1400,
                      normalize_plurals=False,
                      max_words=NUM_PALABRAS_NUBE
                     ).generate_from_frequencies(frecuencias)
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

Volvamos a generar la nube de palabras del mismo usuario de antes, y comparemos el resultado.

In [ ]:
print("Nube de palabras de",usuario)
muestra_word_cloud_por_frecuencias(importancia_usuario, palabras_huecas)